In [4]:
# Librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import lightgbm
import re

from scripts import fe_scripts as fe
from scripts import light
from sklearn.model_selection import train_test_split

# Options
pd.set_option('display.max_columns', 200)
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Feature Engineering on each dataset

## Prepare and store engineered datasets

In [2]:
# Bureau and bureau balance
df_bureau = fe.bureau_and_balance()
df_bureau.to_pickle('../data/data_outputs/feature_engineered/bureau.pkl')
df_bureau.shape

(305811, 79)

In [3]:
# Previous applications
df_previous = fe.previous_applications()
df_previous.to_pickle('../data/data_outputs/feature_engineered/previous.pkl')
df_previous.shape

(338857, 134)

In [4]:
# POS_CASH_balance
df_pos = fe.pos_cash()
df_pos.to_pickle('../data/data_outputs/feature_engineered/pos.pkl')
df_pos.shape

(337252, 18)

In [5]:
# Installments
df_install = fe.installments_payments()
df_install.to_pickle('../data/data_outputs/feature_engineered/installments.pkl')
df_install.shape

(339587, 22)

In [6]:
# Credit card balance
df_ccb = fe.credit_card_balance()
df_ccb.to_pickle('../data/data_outputs/feature_engineered/credit_card.pkl')
df_ccb.shape

(103558, 113)

## Merging all dataframes

In [7]:
# Main dataset
df_app = pd.read_pickle('../data/data_outputs/feature_engineered/application.pkl')
df_app = df_app[~df_app['TARGET'].isna()]
df_app.shape

(307507, 184)

In [8]:
# Merge bureau
df_ = df_app.join(df_bureau, how = 'left', on = 'SK_ID_CURR')

del df_app
del df_bureau
gc.collect()

48

In [9]:
# Merge Previous applications
df_ = df_.join(df_previous, how = 'left', on = 'SK_ID_CURR')
del df_previous
gc.collect()

0

In [10]:
# Merge POS_CASH_balance
df_ = df_.join(df_pos, how = 'left', on = 'SK_ID_CURR')
del df_pos
gc.collect()

0

In [11]:
# Merge installments
df_ = df_.join(df_install, how = 'left', on = 'SK_ID_CURR')
del df_install
gc.collect()

0

In [12]:
# Merge Credit card balance
df_ = df_.join(df_ccb, how = 'left', on = 'SK_ID_CURR')
del df_ccb
gc.collect()

0

In [13]:
df_.shape

(307507, 550)

## Storing dataset

In [ ]:
df_.to_pickle('../data/data_outputs/feature_engineered/full_dataset.pkl')
del df_
gc.collect

# Feature Selection

https://www.kaggle.com/willkoehrsen/introduction-to-feature-selection/notebook

A chaque étape, évaluer l'impact sur les perfs pour ne pas supprimer de features déterminantes, voir faire varier le seuil (R²max / % NaN)

In [5]:
# Loading dataset
data = pd.read_pickle('../data/data_outputs/feature_engineered/full_dataset.pkl')
data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x), inplace = True)

In [6]:
# Baseline performance before feature selection
light.kfold_lightgbm(data.iloc[:10000], num_folds=5, stratified = True)

Starting LightGBM. Train shape: (10000, 550), test shape: (0, 550)
Training until validation scores don't improve for 200 rounds
[100]	training's auc: 0.972319	training's binary_logloss: 0.153493	valid_1's auc: 0.77953	valid_1's binary_logloss: 0.235573
[200]	training's auc: 0.997858	training's binary_logloss: 0.103941	valid_1's auc: 0.781187	valid_1's binary_logloss: 0.234532
[300]	training's auc: 0.999871	training's binary_logloss: 0.0753044	valid_1's auc: 0.775403	valid_1's binary_logloss: 0.238193
Early stopping, best iteration is:
[127]	training's auc: 0.985838	training's binary_logloss: 0.13677	valid_1's auc: 0.782254	valid_1's binary_logloss: 0.233938
Fold  1 AUC : 0.782254
Training until validation scores don't improve for 200 rounds
[100]	training's auc: 0.975841	training's binary_logloss: 0.152654	valid_1's auc: 0.766123	valid_1's binary_logloss: 0.236139
[200]	training's auc: 0.997472	training's binary_logloss: 0.104334	valid_1's auc: 0.778058	valid_1's binary_logloss: 0.233

## Searching and dropping columns high correlations

In [8]:
# Correlation matrix
corr = data.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,...,CC_AMT_DRAWINGS_ATM_CURRENT_MEAN,CC_AMT_DRAWINGS_ATM_CURRENT_SUM,CC_AMT_DRAWINGS_ATM_CURRENT_VAR,CC_AMT_DRAWINGS_CURRENT_MAX,CC_AMT_DRAWINGS_CURRENT_MEAN,CC_AMT_DRAWINGS_CURRENT_SUM,CC_AMT_DRAWINGS_CURRENT_VAR,CC_AMT_DRAWINGS_OTHER_CURRENT_MAX,CC_AMT_DRAWINGS_OTHER_CURRENT_MEAN,CC_AMT_DRAWINGS_OTHER_CURRENT_SUM,CC_AMT_DRAWINGS_OTHER_CURRENT_VAR,CC_AMT_DRAWINGS_POS_CURRENT_MAX,CC_AMT_DRAWINGS_POS_CURRENT_MEAN,CC_AMT_DRAWINGS_POS_CURRENT_SUM,CC_AMT_DRAWINGS_POS_CURRENT_VAR,CC_AMT_INST_MIN_REGULARITY_MAX,CC_AMT_INST_MIN_REGULARITY_MEAN,CC_AMT_INST_MIN_REGULARITY_SUM,CC_AMT_INST_MIN_REGULARITY_VAR,CC_AMT_PAYMENT_CURRENT_MAX,CC_AMT_PAYMENT_CURRENT_MEAN,CC_AMT_PAYMENT_CURRENT_SUM,CC_AMT_PAYMENT_CURRENT_VAR,CC_AMT_PAYMENT_TOTAL_CURRENT_MAX,CC_AMT_PAYMENT_TOTAL_CURRENT_MEAN,CC_AMT_PAYMENT_TOTAL_CURRENT_SUM,CC_AMT_PAYMENT_TOTAL_CURRENT_VAR,CC_AMT_RECEIVABLE_PRINCIPAL_MAX,CC_AMT_RECEIVABLE_PRINCIPAL_MEAN,CC_AMT_RECEIVABLE_PRINCIPAL_SUM,CC_AMT_RECEIVABLE_PRINCIPAL_VAR,CC_AMT_RECIVABLE_MAX,CC_AMT_RECIVABLE_MEAN,CC_AMT_RECIVABLE_SUM,CC_AMT_RECIVABLE_VAR,CC_AMT_TOTAL_RECEIVABLE_MAX,CC_AMT_TOTAL_RECEIVABLE_MEAN,CC_AMT_TOTAL_RECEIVABLE_SUM,CC_AMT_TOTAL_RECEIVABLE_VAR,CC_CNT_DRAWINGS_ATM_CURRENT_MAX,CC_CNT_DRAWINGS_ATM_CURRENT_MEAN,CC_CNT_DRAWINGS_ATM_CURRENT_SUM,CC_CNT_DRAWINGS_ATM_CURRENT_VAR,CC_CNT_DRAWINGS_CURRENT_MAX,CC_CNT_DRAWINGS_CURRENT_MEAN,CC_CNT_DRAWINGS_CURRENT_SUM,CC_CNT_DRAWINGS_CURRENT_VAR,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX,CC_CNT_DRAWINGS_OTHER_CURRENT_MEAN,CC_CNT_DRAWINGS_OTHER_CURRENT_SUM,CC_CNT_DRAWINGS_OTHER_CURRENT_VAR,CC_CNT_DRAWINGS_POS_CURRENT_MAX,CC_CNT_DRAWINGS_POS_CURRENT_MEAN,CC_CNT_DRAWINGS_POS_CURRENT_SUM,CC_CNT_DRAWINGS_POS_CURRENT_VAR,CC_CNT_INSTALMENT_MATURE_CUM_MAX,CC_CNT_INSTALMENT_MATURE_CUM_MEAN,CC_CNT_INSTALMENT_MATURE_CUM_SUM,CC_CNT_INSTALMENT_MATURE_CUM_VAR,CC_SK_DPD_MAX,CC_SK_DPD_MEAN,CC_SK_DPD_SUM,CC_SK_DPD_VAR,CC_SK_DPD_DEF_MAX,CC_SK_DPD_DEF_MEAN,CC_SK_DPD_DEF_SUM,CC_SK_DPD_DEF_VAR,CC_NAME_CONTRACT_STATUS_Active_MAX,CC_NAME_CONTRACT_STATUS_Active_MEAN,CC_NAME_CONTRACT_STATUS_Active_SUM,CC_NAME_CONTRACT_STATUS_Active_VAR,CC_NAME_CONTRACT_STATUS_Approved_MAX,CC_NAME_CONTRACT_STATUS_Approved_MEAN,CC_NAME_CONTRACT_STATUS_Approved_SUM,CC_NAME_CONTRACT_STATUS_Approved_VAR,CC_NAME_CON

In [9]:
# Number and proportion of columns with high correlations 
l_drop = [c for c in upper.columns if any(upper[c]>0.9)]
len(l_drop), len(l_drop)/len(upper)

(149, 0.27090909090909093)

In [11]:
# Performance impact if dropping high correlations
light.kfold_lightgbm(data.iloc[:10000].drop(columns = l_drop), num_folds=5, stratified = True)

Starting LightGBM. Train shape: (10000, 401), test shape: (0, 401)
Training until validation scores don't improve for 200 rounds
[100]	training's auc: 0.973038	training's binary_logloss: 0.155478	valid_1's auc: 0.779764	valid_1's binary_logloss: 0.235497
[200]	training's auc: 0.998409	training's binary_logloss: 0.105343	valid_1's auc: 0.784719	valid_1's binary_logloss: 0.232305
[300]	training's auc: 0.999907	training's binary_logloss: 0.0765227	valid_1's auc: 0.782247	valid_1's binary_logloss: 0.235132
[400]	training's auc: 0.999997	training's binary_logloss: 0.0573394	valid_1's auc: 0.778376	valid_1's binary_logloss: 0.241482
Early stopping, best iteration is:
[201]	training's auc: 0.998445	training's binary_logloss: 0.104989	valid_1's auc: 0.784635	valid_1's binary_logloss: 0.232287
Fold  1 AUC : 0.784635
Training until validation scores don't improve for 200 rounds
[100]	training's auc: 0.977525	training's binary_logloss: 0.154282	valid_1's auc: 0.79058	valid_1's binary_logloss: 0.2

In [ ]:
# Delete columns highly correlated with another column
data = data.drop(columns = l_drop)

In [ ]:
del corr, upper

## Handling missing values

In [ ]:
# Proportion of missing values
s_ = data.isnull().sum()/len(data)
s_.plot(kind = 'hist', bins = 100, figsize = (10,5), cumulative = True)
plt.grid()
plt.show()

In [ ]:
# Number and proportion of columns high percentage of missing values
len(s_[s_>0.75]), len(s_[s_>0.75])/len(s_)

In [ ]:
# Dropping columns where more than 75% of values are missing
l_drop = [x for x in s_[s_>0.75].index]
data = data.drop(columns = l_drop)
data.shape

del s_, l_drop

## Feature importance

In [ ]:
data_sample = data.sample(50000).drop(columns = ['SK_ID_CURR', 'source'])
X_ = data_sample.drop(columns = ['TARGET']).values
y_ = data_sample['TARGET'].values

feature_importances = np.zeros(X_.shape[1])

# LightGBM model
model = lightgbm.LGBMClassifier(objective='binary', 
                                boosting_type = 'goss', 
                                n_estimators = 1000, 
                                class_weight = 'balanced')

# Train the model several times and retrieve mean feature importances
for i in range(2):
    x_train, x_test, y_train, y_test = train_test_split(X_, y_, test_size = 0.35, random_state = i)
    model.fit(x_train, y_train, 
              early_stopping_rounds = 100, 
              eval_set = [(x_test, y_test)],
              eval_metric = 'auc',
              verbose = 200
             )
    feature_importances += model.feature_importances_
    
# Mean of feature_importances
feature_importances = feature_importances/2

In [ ]:
# Feature importance per feature
df_fi = pd.DataFrame(feature_importances,index = data_sample.drop(columns = ['TARGET']).columns, columns = ['FI'])
df_fi.plot(kind = 'hist', cumulative = True, density = True, bins = 100)
plt.show()

In [ ]:
# Features with a null importance
l_drop = list(df_fi[df_fi['FI']==0].index)
len(l_drop), len(l_drop)/data.shape[1]

In [ ]:
# Dropping columns with a null feature importance
data = data.drop(columns = l_drop)
data.shape

In [ ]:
del data_sample, X_, y_, x_train, x_test, y_train, y_test

# Storing cleaned dataset

In [ ]:
data.to_pickle('../data/data_outputs/feature_engineered/cleaned_dataset.pkl')